In [ ]:
import sys
sys.path.insert(0, '/home/yymao/Dropbox/Academia/Collaborations/SAGA-local/saga')

In [ ]:
from pathlib import Path

import SAGA
from SAGA import ObjectCuts as C
from SAGA import HostCuts as H
from easyquery import *
from SAGA.utils import *
from astropy.table import vstack, Table, join
from astropy.coordinates import Distance, SkyCoord

import numexpr as ne
from scipy.stats import binned_statistic
from bottleneck import nanmax, nanmin
from itertools import product

print(SAGA.__version__)

In [ ]:
saga = SAGA.QuickStart()

In [ ]:
base = SAGA.database.FitsTable("/home/yymao/Downloads/saga_base_all.fits").read()

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10.5, 4))


t = (~Query(C.faint_end_limit, C.high_priority_cuts)).filter(base, ["r_mag", "sb_r", "gr"])

ax[0].scatter(t["r_mag"], t["sb_r"], c='grey', s=0.1, lw=0, rasterized=True)
ax[1].scatter(t["r_mag"], t["gr"], c='grey', s=0.1, lw=0, rasterized=True, label='Non-target galaxies')


t = Query(C.faint_end_limit, C.high_priority_cuts, ~C.is_sat).filter(base, ["r_mag", "sb_r", "gr"])

ax[0].scatter(t["r_mag"], t["sb_r"], c='C0', s=1, lw=0, rasterized=True)
ax[1].scatter(t["r_mag"], t["gr"], c='C0', s=1, lw=0, rasterized=True, label='Target galaxies')


r = np.linspace(12, 21, 10)
ax[0].plot(r, 0.6*(r-14) + 17.8, c='C2', ls="--")
ax[1].plot(r, -0.06*(r-14) + 0.9, c='C2', ls="--", label='Targeting cuts')
ax[1].axhline(0.85, c='C2', ls="--", alpha=0.5)
#ax[1].text(20.75, 0.88, 'Paper I $gr$ cut', ha="right", c='k')

t = Query(C.is_sat, saga.host_catalog.construct_host_query("paper1")).filter(base, ["r_mag", "sb_r", "gr"])

ax[0].scatter(t["r_mag"], t["sb_r"], c='C1', s=18, lw=0, rasterized=True, marker="^", label='Satellites in Paper I')
ax[1].scatter(t["r_mag"], t["gr"], c='C1', s=18, lw=0, rasterized=True, marker="^")

t = Query(C.is_sat, ~saga.host_catalog.construct_host_query("paper1")).filter(base, ["r_mag", "sb_r", "gr"])

ax[0].scatter(t["r_mag"], t["sb_r"], c='C3', s=10, lw=0, rasterized=True, label='Satellites (this work)', marker="s")
ax[1].scatter(t["r_mag"], t["gr"], c='C3', s=10, lw=0, rasterized=True, marker="s")

ax[0].set_xlim(12, 21)
ax[1].set_xlim(12, 21)

ax[0].set_ylim(16, 26)
ax[1].set_ylim(-0.3, 2)

ax[1].set_xlabel(r'$r_o$')
ax[1].set_ylabel(r'$(g-r)_o$')
ax[0].set_xlabel(r'$r_o$')
ax[0].set_ylabel(r'${\mu}_{r_o,{\rm eff}}$')

lgnd = ax[0].legend(loc="lower right", scatterpoints=3, frameon=True)
lgnd.legendHandles[0]._sizes = [30]
lgnd.legendHandles[1]._sizes = [20]

lgnd = ax[1].legend(loc="upper left", scatterpoints=3, frameon=True)
lgnd.legendHandles[1]._sizes = [10]
lgnd.legendHandles[2]._sizes = [20]

fig.tight_layout()

#fig.colorbar(c, cax=fig.add_axes((0.90, 0.52, 0.01, 0.4)), label=r'$p_{\rm GMM}$')
plt.savefig('/home/yymao/Downloads/targeting-cuts.pdf', dpi=200)

In [ ]:
specs = Query(C.basic_cut2, C.has_spec, "SPEC_Z > 0.003", saga.host_catalog.construct_host_query("need_spec < 50", "paper2_complete"), "SATS != 3").filter(base)

In [ ]:
len(specs) / Query(C.basic_cut2, saga.host_catalog.construct_host_query("need_spec < 50", "paper2_complete")).count(base)

In [ ]:
hosts = np.unique(specs["HOSTID"])
print(len(hosts))
total_sq_deg = (np.rad2deg(np.arcsin(0.3 / saga.host_catalog.load(hosts)["DIST"]))**2 * np.pi).sum()
print(total_sq_deg)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10.5,3.5))

for ax_this, zbins in zip(ax, [np.linspace(0, 0.8, 33), np.linspace(0, 0.05, 11)]):
    zbins[0] = 0.003
    for q, kw in (
        (Query(), dict(label='All redshifts', alpha=0.25, color="C1")),
        (C.gri_cut, dict(label='Paper I $gri$ cuts', alpha=0.4, color="C2")),
        (C.high_priority_cuts, dict(label='Targeting cuts (this work)', color='C2', alpha=0.95)),
    ):
        ax_this.hist(q.filter(specs, 'SPEC_Z'), zbins, weights=np.ones(q.count(specs))/total_sq_deg, **kw)

        ax_this.set_xlabel(r'$z_{\rm spec}$')
        ax_this.set_ylabel(r'$N_{\rm spec}$ (per sq. deg., per $z$ bin)')
        ax_this.set_xlim(0, 0.8 if zbins[-1] == 0.8 else 0.052)
        
ax[0].legend(loc='upper right', markerfirst=False, handletextpad=0.1)
plt.tight_layout()
plt.savefig('/home/yymao/Downloads/z_distribution.pdf', bbox_inches='tight')

In [ ]:
base1 = Query(saga.host_catalog.construct_host_query("paper2_complete")).filter(base)

In [ ]:
def add_scores(specs):
    specs["score_sb_r"] = ne.evaluate("sb_r + abs(sb_r_err) - 0.6 * (r_mag - 14)", specs, {})
    for c in ("gr", "ri", "rz"):
        specs["score_{}_r".format(c)] = ne.evaluate("{0} - abs({0}_err) + 0.06* (r_mag - 14)".format(c), specs, {})
    return specs

base1 = add_scores(base1)

In [ ]:

fig, ax = plt.subplots(ncols=3, figsize=(10.5, 4))
for i, ax_this in enumerate(ax):
    
    if i == 0:
        key = "r_mag"
        ax_this.set_xlabel(r'$r_o$')
        bins = np.linspace(11, 21, 21)
        bins[-1] = 20.75
    elif i == 1:
        key = "score_sb_r"
        ax_this.set_xlabel(r'$\mu_{r_o,{\rm eff}} + \sigma_\mu + 0.6\,(r_o-14)$')
        bins = np.linspace(14, 24, 21)
        cut_other = Query(C.faint_end_limit, C.high_priority_gr, C.high_priority_ri, C.high_priority_rz, C.high_priority_ug)
    elif i == 2:
        key = "score_gr_r"
        ax_this.set_xlabel(r'$(g-r)_o - \sigma_{gr} - 0.06\,(r_o - 14) $')
        bins = np.linspace(0, 2, 21)
        cut_other = Query(C.faint_end_limit, C.high_priority_sb)
        
    bin_index = np.digitize(base1[key], bins)
    cut = Query(C.faint_end_limit, C.high_priority_cuts)
    
    x = bins[:-1]
    width = np.ediff1d(bins)
    def add_bar(*queries, **kwargs):
        y = np.maximum(np.bincount(bin_index[Query(*queries).mask(base1)], minlength=len(bins)+1)[1:-1], 0.01) 
        return ax_this.bar(x, y, width=width, align='edge', **kwargs)

    c_value = 0.35
    l0 = add_bar(color=plt.cm.Greys(c_value))
    __ = add_bar(C.has_spec, color=plt.cm.Blues(c_value))
    __ = add_bar(C.has_spec, C.has_our_specs_only, color=plt.cm.Greens(c_value))
    
    if False:
        c_value = 0.5
        __ = add_bar(cut_other, color=plt.cm.Greys(c_value))
        __ = add_bar(cut_other, C.has_spec, color=plt.cm.Blues(c_value))
        __ = add_bar(cut_other, C.has_spec, C.has_our_specs_only, color=plt.cm.Greens(c_value))
    
    c_value = 0.65
    l1 = add_bar(cut, color=plt.cm.Greys(c_value))
    l2 = add_bar(cut, C.has_spec, color=plt.cm.Blues(c_value))
    l3 = add_bar(cut, C.has_spec, C.has_our_specs_only, color=plt.cm.Greens(c_value))
    l4 = add_bar(cut, C.has_spec, C.is_sat, color=plt.cm.Oranges(c_value))
    
    ax_this.set_yscale("symlog", linthreshy=10, subsy=[2,3,4,5,6,7,8,9], linscaley=np.log10(2))
    ax_this.set_ylim(0, 3e4)
    ax_this.grid("on", axis="y", color="k", lw=0.5)
    ax_this.set_xlim(bins[0], bins[-1])
    if i == 0:
        bins[-1] = 21
    ax_this.set_xticks(bins[::5] if i == 2 else bins[::4])
    

ax[0].set_ylabel(r'$N_{\rm galaxy}$ (per bin)')

x0, y0 = 12, 2e3
for mult in (1.1, 2, 5):
    ax[0].plot([x0, x0], [y0, y0*mult], c='k', lw=2, marker='')
    ax[0].text(x0, y0*0.88, fr'$\times\,{mult}$', ha="center", va="top")
    x0 += 1.8

plt.tight_layout(rect=(0,0,1,0.9), w_pad=0.1)

plt.figlegend((l1, l0), ("Targets (within targeting cuts)", "Non-targets (outside within targeting cuts)"), loc="upper right", markerfirst=False, bbox_to_anchor=(0.955,1))
plt.figlegend((l1, l2, l3, l4), ("No redshift", "Pre-existing redshift", "SAGA redshift", "Satellites"), loc="upper left", ncol=2, bbox_to_anchor=(0.065,1))

plt.savefig('/home/yymao/Downloads/target_spec_counts.pdf', bbox_inches='tight')